<a href="https://colab.research.google.com/github/ffg-kom/mapping_evolution/blob/master/getting_data_ready.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

The following code was copied from https://colab.research.google.com/github/nestauk/im_tutorials/blob/master/notebooks/01_intro_to_pandas_tutorial.ipynb.

In [0]:
%load_ext autoreload
%autoreload 2

# install im_tutorial package
!pip install git+https://github.com/nestauk/im_tutorials.git

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
  Cloning https://github.com/nestauk/im_tutorials.git to /tmp/pip-req-build-xa9si820
  Running command git clone -q https://github.com/nestauk/im_tutorials.git /tmp/pip-req-build-xa9si820
  Created wheel for im-tutorials: filename=im_tutorials-0.1.1-cp36-none-any.whl size=15103 sha256=f92906c31322f3acfd5ea3f784af47d8cf37d2b6b66315a52dba0bf5cf39d878
  Stored in directory: /tmp/pip-ephem-wheel-cache-4uz_o3mg/wheels/47/a3/cb/bdc5f9ba49bcfd2c6864b166a1566eb2f104113bf0c3500330
Successfully built im-tutorials


In [0]:
# numpy for mathematical functions
import numpy as np
# pandas for handling tabular data
import pandas as pd
# explained later
from im_tutorials.data.cordis import h2020_projects

# for plotting
import matplotlib.pyplot as plt

# data
from im_tutorials.data.cordis import cordis_table

# Loading data

In [203]:
# load tables
cordis_o_df = cordis_table('organisations')
cordis_p_df = cordis_table('projects')
cordis_p_o_df = cordis_table('project_organisations')
cordis_pub_df = cordis_table('publications')

# merge them
cordis_project_orgs_df = cordis_p_o_df.merge(
  cordis_o_df, left_on='organization_id', right_on='id', how='left'
)
cordis_project_orgs_df = cordis_project_orgs_df.merge(
  cordis_p_df, left_on='project_rcn', right_on='rcn', how='left'
)

cordis_project_orgs_df.rename( columns={'contribution': 'ec_contribution_organization'
                                        , 'ec_contribution': 'ec_contribution_project'
                                        , 'total_cost': 'total_cost_project'
                                       }, inplace=True )

cordis_project_orgs_df.shape

(251040, 25)

In [204]:
# add number of partners per project as a column
count_partners_df = cordis_project_orgs_df[['project_rcn', 'organization_id']].groupby( by='project_rcn' ).count()
count_partners_df.rename( columns={'organization_id': 'number_of_partners'}, inplace=True )

cordis_project_orgs_df = cordis_project_orgs_df.merge(
  count_partners_df, on='project_rcn', how='left'
)

# add number of publications per project as a column
count_publications_df = cordis_pub_df[['id', 'project_rcn']].groupby( by='project_rcn' ).count()
count_publications_df.rename( columns={'id': 'number_of_publications'}, inplace=True )

cordis_project_orgs_df = cordis_project_orgs_df.merge(
  count_publications_df, on='project_rcn', how='left'
)


# shape: should by (something, 27) 
cordis_project_orgs_df.shape


(251040, 27)

In [205]:
# narrow our dataset down to what we want to work with
subset = cordis_project_orgs_df[
    (cordis_project_orgs_df.activity_type == 'Private for-profit entities (excluding Higher or Secondary Education Establishments)')
  & (cordis_project_orgs_df.framework == 'H2020')
]

df = subset[['organization_id', 'start_date_code', 'funding_scheme', 'number_of_partners', 'number_of_publications', 'ec_contribution_organization', 'ec_contribution_project', 'total_cost_project']]
df.shape

(37138, 8)


# Playing with the data

(No need to to run anything from in here when preparing data for export.)

In [0]:
cordis_project_orgs_df.columns

Index(['project_rcn', 'organization_id', 'activity_type', 'address',
       'ec_contribution_organization', 'type', 'website_x', 'id', 'name',
       'country_code', 'country_name', 'rcn', 'acronym', 'end_date_code',
       'ec_contribution_project', 'framework', 'funding_scheme',
       'funded_under', 'objective', 'project_description', 'start_date_code',
       'status', 'title', 'total_cost_project', 'website_y',
       'number_of_partners'],
      dtype='object')

In [0]:
cordis_project_orgs_df['activity_type'].value_counts()

Higher or Secondary Education Establishments                                                         88335
Private for-profit entities (excluding Higher or Secondary Education Establishments)                 78810
Research Organisations                                                                               55099
Other                                                                                                15406
Public bodies (excluding Research Organisations and Secondary or Higher Education Establishments)    12454
                                                                                                       936
Name: activity_type, dtype: int64

In [0]:
# within subset, how often do organisations appear
subset['organization_id'].value_counts().head()



999993856    148
999960488     84
999909854     79
999951467     76
999908787     73
Name: organization_id, dtype: int64

In [0]:
subset[['organization_id', 'start_date_code', 'funding_scheme', 'number_of_partners', 'ec_contribution_organization', 'ec_contribution_project', 'total_cost_project']]


In [0]:
for i in range(100):
  print(cordis_project_orgs_df.loc[i + 10000].funded_under[0]['title'])



# Preparing data for a Sankey diagram




In [208]:
# putting data in order
df = df.sort_values( by=['organization_id', 'start_date_code'] ).reset_index()

# add a column that numbers entries per organisation
df['organization_entry_number'] = df.groupby('organization_id').cumcount()

df[['organization_id', 'start_date_code', 'organization_entry_number']].head()

,organization_id,start_date_code,organization_entry_number
0,888888310,2016-12-01,0
1,888888341,2018-01-01,0
2,888888620,2017-09-01,0
3,888888655,2017-03-01,0
4,888888707,2016-09-01,0


In [215]:
# looking at first two entries of organisations
first = df[df['organization_entry_number'] == 0]
second = df[df['organization_entry_number'] == 1]

onestep = second.merge(first, on='organization_id', how='left')


# aggregate over funding schemes
sankey_df = onestep[['funding_scheme_y', 'funding_scheme_x', 'organization_id']].groupby(['funding_scheme_y', 'funding_scheme_x']).count()

# prepare for proper output (index -> columns, renaming)
sankey_df.reset_index(level=['funding_scheme_y', 'funding_scheme_x'], inplace=True)
sankey_df.rename(columns={'funding_scheme_y':'source', 'funding_scheme_x':'target'}, inplace=True)

# add a blank space to all targets (so targets differ from sources)
sankey_df['target'] = sankey_df['target'] + ' '

sankey_df.shape

(368, 3)

In [0]:
# export csv
sankey_df.to_csv('sankey_onestep_data.csv') 
